In [1]:
import requests
import re
from bs4 import BeautifulSoup
from googlesearch import search

TownCharts.com will provide the initial set of variables.
Other variables might be merged after ...

First collect relevant variables from each figures that display consistent information from each of the tabs from San Luis Obispo: Demographics, Housing,Economy,Education,Healthcare. 

And then automate since its a structured website for cities from major metropolitan areas. 

First up is the housing the variables. Out of the 44 figures 18(3,9,13,16,19,20,22,23,25,26,27,31,32,33,34,35,38,39) will not be included due to incosistencies. Some cities might not have the figure information and an NA will be put in place. 

Note figure 24:will have several variables


In [2]:
url = 'https://www.towncharts.com/California/Housing/San-Luis-Obispo-city-CA-Housing-data.html'
r = requests.get(url)
soup = BeautifulSoup(r.text,'html.parser')



# **Note:Each housing tab has 3 vital sections in total**

In [65]:
def get_text_section(target_section):
    found = soup.find_all('div',attrs = {'class':target_section})
    validsection = found[0]
    text_section = validsection.find_all(text=True)
    return text_section
housing_text_section1 = get_text_section('section1')
housing_text_section2 = get_text_section('section2')
housing_text_section3 = get_text_section('section4')

Data dictionary:
<br>
    housing_units: total number of housing units for the city
    <br>
    housing_density: housing density in the area as measured     by houses per square land mile
    change_hunits: the change in housing units
    

In [39]:
def searchpattern(pattern,resultset):
    '''
    pattern:regex pattern
    resultset: A beautiful soup result set
    output housing variable of interest type string
    '''
    result = [i for i in resultset if re.search(pattern,i)]
    if result == []:
        result  = None
    else:
        result = re.findall(pattern,result[0])
    return result

#for variables need to remove everything but numbers
#figure1 
housing_units = searchpattern('it\shas\s(\d.*?\d)\shousing\sunits\s',housing_text_section1)
#figure2 
housing_density = searchpattern('it\shas\s(\d.*?\d)\shousing\sdensity\s',housing_text_section1)
#figure4 
change_hunits = searchpattern('it\shas\s(.*?%)\schange\sin\s',housing_text_section1)
#figure5
percent_of_rent_to_total = searchpattern('it\shas\s(\d.*?%)\sRenter\sPercent\s',housing_text_section1)
#figure 6
owned_homes = searchpattern('it\shas\s(\d.*?%)\sOwner\sPercent\s',housing_text_section1)
#figure 7 median home value
med_homeval = searchpattern('it\shas\s(.*?\d)\smedian\shome\svalue\s',housing_text_section2)
#figure 8 median rental rate
med_rental_rate = searchpattern('it\shas\s(.*?\d)\srental\srates\s',housing_text_section2)
#figure 10 median owner costs
med_owner_cost = searchpattern("it\shas\s(.*?\d)\smedian\sownership\scost\s",housing_text_section2)
#figure 11
med_OwnCostas_Perc_of_Income = searchpattern('median\sowner\scost\sas\sa\spercent\sof\stotal\shousehold\sincome\sand\sit\shas\s(\d.*?%)',housing_text_section2)

#figure 12 median home value as a percent of a median workers earnings
med_HCasPerc_of_medearn = searchpattern("it\shas\s(\d.*?%)\smedian\shome\svalue\sas\s",housing_text_section2)
#figure14
med_hcost_ownmortg=searchpattern('depicts.*\$(.*?\d)\smedian',housing_text_section2)
#figure 15
med_hcost_own_wo_mortg = searchpattern('are.*\$(.*?\d)\smedian\shousing\s',housing_text_section2)
#figure 17
hcost_as_perc_of_hincome_ownmortg= searchpattern('with\smortgages.*\s(\d.*?%)\scost',housing_text_section2)
#figure18
hcost_as_perc_of_hincome_womortg= searchpattern('This\sanalysis.*\s(\d.*?%)\scost',housing_text_section2)
#figure 21 **
med_real_estate_taxes =searchpattern('it\shas\s(.*?\d)\smedian\sreal\sestate\s',housing_text_section2)
#figure 24 biggie

#figure 28 format 3.0




In [73]:
searchpattern('it\shas\s(.*?\d)\sfamily\smembers\sper\s',housing_text_section3)

['3.0']

In [56]:
housing_text_section3

['San Luis Obispo city-CA California Home Cost Charts',
 'Looking at the housing data in ',
 'Figure 7',
 ' we start with home values for the area. ',
 'San Luis Obispo city-CA',
 ' shows it has $590,800 median home value which is the 6th in median home value out of 10 total in the area. The city with the highest median home value in the area is ',
 'Los Ranchos',
 ' with a value of $876,000 measures 48.3% larger. ',
 'Figure 8',
 ' shows the median rental rate in the area. San Luis Obispo city-CA shows it has $1,403 rental rates which is the 5th in rental rates out of 10 total in the area. The city with the highest rental rates in the area is ',
 'Santa Margarita',
 ' with a rate of $1,575 measures 12.3% larger. Comparing rental rates to the ',
 'United States',
 ' average of $982, San Luis Obispo city-CA measures 42.9% larger. Also, measured against the state of ',
 'California',
 ', rental rates of $1,358, San Luis Obispo city-CA is only about 3.3% larger. ',
 'Figure 9',
 ' is a fr

In [71]:
testing = [i for i in housing_text_section3 if re.search('it\shas\s(.*?\d)\sfamily\smembers\sper\s',i)]
print(testing)

[' shows the household size for families living in the area and it has 3.0 family members per housing unit which is the 2nd most of all the places in the area. The city with the highest family members per housing unit in the area is ']


In [121]:
# housing_units=[i for i in housing_text_section1 if re.search('it\shas\s(\d.*?\d)\shousing\sunits\s',i)]
# if housing_units == []:
#     housing_units = None
# else:
#     housing_units = re.findall('it\shas\s(\d.*?\d)\shousing\sunits\s',housing_units[0])
    
    
        
# print(housing_units)
# housing_change = [i for i in housing_text_section1 if re.search('it\shas\s(.*?%)\schange\sin\s',i)]
# h_change = re.findall('it\shas\s(.*?%)\schange\sin\s',housing_change[0])
# print(h_change)
# Percent_of_Rent_to_Total = [i for i in housing_text_section1 if re.search('it\shas\s(\d.*?%)\sRenter\sPercent\s',i)]
# print(Percent_of_Rent_to_Total)

['20,332']


#Automation
#list of cities gathered from the major metropolitan areas

In [41]:
city_list = ["San Francico","San Luis Obispo"]


pause : Lapse to wait between HTTP requests. Lapse too short may cause Google to block your IP. Keeping significant lapse will make your program slow but its safe and better option.

In [42]:
results = []
#towncharts Housing san francisco CA
for city in city_list:
    query = "towncharts Housing {} CA".format(city)
    the_url = list(search(query,tld = "co.in",num =1,stop =1,pause =2))[0]
    results.append(the_url)
print(results)

['https://www.towncharts.com/California/Housing/San-Francisco-city-CA-Housing-data.html', 'http://www.towncharts.com/California/Housing/San-Luis-Obispo-County-CA-Housing-data.html']
